# 03 — Calibration Notebook

This notebook analyzes and illustrates the four copula calibration chapters:

1. **Pseudo-MLE (PMLE)**
2. **Inference Functions for Margins (IFM)**
3. **Kendall's τ inversion**
4. **Minimum Distance Estimation (CvM)**

It also **generates and saves** all figures referenced by the chapters to:

```
docs/assets/figures/03_calibration/*.svg
```

> Re-run the cells to regenerate figures. The notebook is self-contained and
> uses synthetic data for clear, reproducible visuals.


In [ ]:
# Setup
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

try:
    import scipy
    from scipy.stats import norm, t as student_t, kendalltau, rankdata, multivariate_normal
    SCIPY = True
except Exception:
    SCIPY = False
    from math import erf
    def norm_cdf(x):  # basic normal CDF fallback
        return 0.5 * (1.0 + erf(x / np.sqrt(2.0)))

ROOT = Path(".")
FIG_DIR = ROOT / "docs" / "assets" / "figures" / "03_calibration"
FIG_DIR.mkdir(parents=True, exist_ok=True)

def savefig(path, tight=True):
    if tight:
        plt.tight_layout()
    plt.savefig(path, format="svg")
    plt.close()

def empirical_copula_uv(x):
    n, d = x.shape
    U = np.zeros_like(x, dtype=float)
    for j in range(d):
        ranks = rankdata(x[:, j], method="ordinal") if SCIPY else np.argsort(np.argsort(x[:, j])) + 1
        U[:, j] = ranks / (n + 1.0)
    return U

def sample_gaussian_copula(n=5000, rho=0.65, seed=7):
    rng = np.random.default_rng(seed)
    cov = np.array([[1.0, rho], [rho, 1.0]])
    z = rng.multivariate_normal(mean=[0, 0], cov=cov, size=n)
    u = norm.cdf(z) if SCIPY else 0.5 * (1.0 + erf(z / np.sqrt(2.0)))
    return u, z

def gaussian_copula_loglikelihood_grid(U, rhos):
    if SCIPY:
        z = norm.ppf(U)
    else:
        eps = 1e-10
        Uc = np.clip(U, eps, 1 - eps)
        # Acklam inverse CDF approximation (see script that created this notebook)
        def inv_norm_cdf(p):
            a = [ -3.969683028665376e+01,  2.209460984245205e+02,
                 -2.759285104469687e+02,  1.383577518672690e+02,
                 -3.066479806614716e+01,  2.506628277459239e+00 ]
            b = [ -5.447609879822406e+01,  1.615858368580409e+02,
                 -1.556989798598866e+02,  6.680131188771972e+01,
                 -1.328068155288572e+01 ]
            c = [ -7.784894002430293e-03, -3.223964580411365e-01,
                 -2.400758277161838e+00, -2.549732539343734e+00,
                  4.374664141464968e+00,  2.938163982698783e+00 ]
            d = [ 7.784695709041462e-03,  3.224671290700398e-01,
                  2.445134137142996e+00,  3.754408661907416e+00 ]
            plow = 0.02425
            phigh = 1 - plow
            q = np.zeros_like(p)
            x = np.zeros_like(p)
            mask = p < plow
            if np.any(mask):
                q = np.sqrt(-2*np.log(p[mask]))
                x[mask] = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
                           ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)
            mask = (p >= plow) & (p <= phigh)
            if np.any(mask):
                q = p[mask] - 0.5
                r = q*q
                x[mask] = (((((a[0]*r + a[1])*r + a[2])*r + a[3])*r + a[4])*r + a[5])*q / \
                           (((((b[0]*r + b[1])*r + b[2])*r + b[3])*r + b[4])*r + 1)
            mask = p > phigh
            if np.any(mask):
                q = np.sqrt(-2*np.log(1 - p[mask]))
                x[mask] = -(((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
                            ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)
            return x
        z = inv_norm_cdf(Uc)
    z1, z2 = z[:, 0], z[:, 1]
    L = []
    for rho in rhos:
        det = 1 - rho**2
        inv = 1.0 / det
        quad = (z1**2 - 2*rho*z1*z2 + z2**2) * inv
        logc = -0.5*np.log(det) - 0.5*(quad - (z1**2 + z2**2))
        L.append(np.sum(logc))
    return np.array(L)

def grid2d(n=60):
    u = np.linspace(0.001, 0.999, n)
    U, V = np.meshgrid(u, u, indexing="xy")
    return U, V

def gaussian_copula_cdf(u, v, rho):
    if SCIPY:
        from scipy.stats import multivariate_normal
        x = norm.ppf(u)
        y = norm.ppf(v)
        pts = np.column_stack([x.ravel(), y.ravel()])
        vals = multivariate_normal(mean=[0,0], cov=[[1,rho],[rho,1]]).cdf(pts)
        return vals.reshape(u.shape)
    return u * v  # fallback

def empirical_copula_surface(U, ngrid=60):
    Uu, Vv = grid2d(ngrid)
    n = U.shape[0]
    A = (U[:, 0][:, None, None] <= Uu[None, :, :]) & (U[:, 1][:, None, None] <= Vv[None, :, :])
    Cn = A.mean(axis=0)
    return Uu, Vv, Cn

# Synthetic dataset (shared)
n = 5000
rho_true = 0.65
U_data, Z_data = sample_gaussian_copula(n=n, rho=rho_true, seed=7)
U_pseudo = empirical_copula_uv(Z_data)


## 1) Pseudo-MLE (PMLE)

We inspect the log-likelihood shape for the Gaussian copula, visualize
pseudo-observations, and show an illustrative optimization trace.


In [ ]:
# PMLE — figures
rhos = np.linspace(-0.95, 0.95, 181)
L = gaussian_copula_loglikelihood_grid(U_pseudo, rhos)

plt.figure(figsize=(6, 4))
plt.plot(rhos, L); plt.axvline(rho_true, linestyle="--")
plt.xlabel("rho"); plt.ylabel("log-likelihood (up to const)")
plt.title("Gaussian Copula — Pseudo-MLE log-likelihood over rho")
savefig(FIG_DIR / "pseudo_mle_gaussian_surface.svg")

plt.figure(figsize=(5, 5))
plt.scatter(U_pseudo[:,0], U_pseudo[:,1], s=3, alpha=0.35)
plt.xlabel("u"); plt.ylabel("v"); plt.title("Pseudo-observations (rank uniforms)")
plt.gca().set_aspect('equal', adjustable='box')
savefig(FIG_DIR / "pseudo_mle_pseudoobs.svg")

trace = []
rho = 0.0; lr = 0.25
for _ in range(20):
    eps = 1e-3
    l0 = gaussian_copula_loglikelihood_grid(U_pseudo, np.array([rho]))[0]
    l1 = gaussian_copula_loglikelihood_grid(U_pseudo, np.array([rho + eps]))[0]
    l2 = gaussian_copula_loglikelihood_grid(U_pseudo, np.array([rho - eps]))[0]
    grad = (l1 - l2) / (2 * eps)
    rho = np.clip(rho + lr * grad / (1 + abs(grad)), -0.95, 0.95)
    trace.append((rho, l0))

plt.figure(figsize=(6, 4))
plt.plot([t[1] for t in trace], marker="o")
plt.title("PMLE Optimization Trace (illustrative)")
plt.xlabel("iteration"); plt.ylabel("log-likelihood")
savefig(FIG_DIR / "pseudo_mle_optimization_trace.svg")

## 2) Inference Functions for Margins (IFM)

We illustrate the two-step IFM pipeline and compare the Gaussian copula
log-likelihood with PMLE.


In [ ]:
# IFM — pipeline + surfaces
# Pipeline schematic
plt.figure(figsize=(7, 3.2))
ax = plt.gca(); ax.axis('off'); ax.set_xlim(0,10); ax.set_ylim(0,5)
boxes = [(0.5,3.0,"Raw data\n$(X_1,\\ldots,X_d)$"),
         (3.0,3.0,"Fit marginals\n$F_i(x;\\hat\\phi_i)$"),
         (5.5,3.0,"Transform\n$U_i=F_i(X_i;\\hat\\phi_i)$"),
         (8.0,3.0,"Fit copula\n$\\hat\\theta$ via MLE")]
for x,y,text in boxes:
    rect = plt.Rectangle((x,y),1.8,1.2,fill=False); ax.add_patch(rect)
    ax.text(x+0.9, y+0.6, text, ha='center', va='center')
for i in range(len(boxes)-1):
    (x,y,_),(xn,yn,_) = boxes[i], boxes[i+1]
    ax.annotate("", xy=(xn,yn+0.6), xytext=(x+1.8,y+0.6), arrowprops=dict(arrowstyle="->"))
plt.title("IFM estimation pipeline")
savefig(FIG_DIR / "ifm_pipeline.svg")

# Gaussian surface
L_ifm = gaussian_copula_loglikelihood_grid(U_pseudo, rhos)
plt.figure(figsize=(6, 4))
plt.plot(rhos, L_ifm); plt.axvline(rho_true, linestyle="--")
plt.xlabel("rho"); plt.ylabel("log-likelihood (up to const)")
plt.title("IFM — Gaussian copula log-likelihood over rho")
savefig(FIG_DIR / "ifm_gaussian_surface.svg")

# IFM vs PMLE
plt.figure(figsize=(6, 4))
plt.plot(rhos, L, label="PMLE")
plt.plot(rhos, L_ifm, linestyle="--", label="IFM")
plt.xlabel("rho"); plt.ylabel("log-lik (up to const)")
plt.title("IFM vs PMLE (Gaussian copula)"); plt.legend()
savefig(FIG_DIR / "ifm_vs_pseudo_mle.svg")

## 3) Kendall's τ inversion

We plot τ-to-parameter mappings, a small pipeline, and compare τ-inversion
with PMLE on synthetic data.


In [ ]:
# τ inversion — mappings and comparison
tau_vals = np.linspace(-0.95, 0.95, 191)
rho_map = np.sin(0.5 * np.pi * tau_vals)
theta_clayton = 2 * (tau_vals.clip(min=0)) / (1 - tau_vals.clip(min=0))
theta_gumbel = 1 / (1 - tau_vals.clip(min=0.0001))

plt.figure(figsize=(6, 4))
plt.plot(tau_vals, rho_map, label="Gaussian/t: ρ(τ)=sin(πτ/2)")
plt.plot(tau_vals, theta_clayton, label="Clayton: θ(τ)=2τ/(1-τ)")
plt.plot(tau_vals, theta_gumbel, label="Gumbel: θ(τ)=1/(1-τ)")
plt.ylim(-1.1, 6.0)
plt.xlabel("Kendall's τ"); plt.ylabel("Parameter")
plt.title("τ-to-parameter mappings"); plt.legend()
savefig(FIG_DIR / "tau_inversion_mapping.svg")

# Pipeline schematic
plt.figure(figsize=(7, 3.2))
ax = plt.gca(); ax.axis('off'); ax.set_xlim(0,10); ax.set_ylim(0,5)
boxes = [(0.5,3.0,"Data ranks\n$\\hat\\tau$"),
         (3.0,3.0,"Choose copula\n$\\tau(\\theta)$"),
         (5.5,3.0,"Solve\n$\\tau(\\hat\\theta)=\\hat\\tau$"),
         (8.0,3.0,"Use $\\hat\\theta$")]
for x,y,text in boxes:
    rect = plt.Rectangle((x,y),1.8,1.2,fill=False); ax.add_patch(rect)
    ax.text(x+0.9,y+0.6,text,ha='center',va='center')
for i in range(len(boxes)-1):
    (x,y,_),(xn,yn,_) = boxes[i], boxes[i+1]
    ax.annotate("", xy=(xn,yn+0.6), xytext=(x+1.8,y+0.6), arrowprops=dict(arrowstyle="->"))
plt.title("Kendall's τ inversion pipeline")
savefig(FIG_DIR / "tau_inversion_pipeline.svg")

# Compare estimates
if SCIPY:
    tau_emp, _ = kendalltau(U_pseudo[:,0], U_pseudo[:,1])
else:
    idx = np.random.choice(len(U_pseudo), size=min(2000, len(U_pseudo)), replace=False)
    x = U_pseudo[idx,0]; y = U_pseudo[idx,1]
    c = d = 0
    for i in range(len(idx)):
        s = np.sign((x[i] - x[i+1:]) * (y[i] - y[i+1:]))
        c += np.sum(s > 0); d += np.sum(s < 0)
    tau_emp = 2*(c - d) / (len(idx)*(len(idx)-1))

rho_tau = np.sin(0.5*np.pi*tau_emp)
rho_p_mle = rhos[np.argmax(L)]

plt.figure(figsize=(6, 4))
plt.axvline(rho_tau, label=f"τ-inversion ρ≈{rho_tau:.3f}")
plt.axvline(rho_p_mle, linestyle="--", label=f"PMLE ρ≈{rho_p_mle:.3f}")
plt.axvline(rho_true, linestyle="-.", label=f"True ρ={rho_true:.2f}")
plt.xlim(-1.0, 1.0); plt.xlabel("rho"); plt.title("τ-inversion vs PMLE (Gaussian)")
plt.legend()
savefig(FIG_DIR / "tau_inversion_vs_pseudo_mle.svg")

## 4) Minimum Distance Estimation — Cramér–von Mises (CvM)

We visualize the empirical vs model copula surface, the CvM distance
landscape over ρ, and residual diagnostics at the minimizer.


In [ ]:
# CvM — surfaces and residuals
Uu, Vv, Cn = empirical_copula_surface(U_pseudo, ngrid=70)
C_model_true = gaussian_copula_cdf(Uu, Vv, rho_true)

plt.figure(figsize=(5.2, 4.5))
diff = Cn - C_model_true
im = plt.imshow(diff, origin="lower", extent=[0,1,0,1], aspect="equal")
plt.xlabel("u"); plt.ylabel("v"); plt.title("Empirical - Model copula (Gaussian)")
plt.colorbar(im, fraction=0.046, pad=0.04)
savefig(FIG_DIR / "cvm_surface_comparison.svg")

def cvm_distance(U, rho):
    if SCIPY:
        from scipy.stats import multivariate_normal, norm
        x = norm.ppf(U[:,0]); y = norm.ppf(U[:,1])
        pts = np.column_stack([x, y])
        Cvals = multivariate_normal(mean=[0,0], cov=[[1,rho],[rho,1]]).cdf(pts)
    else:
        Cvals = U[:,0] * U[:,1]
    n = len(U)
    from scipy.stats import rankdata as _rank if SCIPY else None
    if SCIPY:
        ru = _rank(U[:,0], method="ordinal"); rv = _rank(U[:,1], method="ordinal")
    else:
        ru = np.argsort(np.argsort(U[:,0])) + 1
        rv = np.argsort(np.argsort(U[:,1])) + 1
    Cemp_at_U = (ru / (n+1)) * (rv / (n+1))
    return np.mean((Cemp_at_U - Cvals)**2)

rhos_dense = np.linspace(-0.9, 0.9, 121)
Sn = np.array([cvm_distance(U_pseudo, r) for r in rhos_dense])

plt.figure(figsize=(6, 4))
plt.plot(rhos_dense, Sn); plt.axvline(rho_true, linestyle="-.", label="True ρ")
plt.xlabel("rho"); plt.ylabel("CvM distance $S_n(\\theta)$")
plt.title("CvM distance landscape (Gaussian copula)"); plt.legend()
savefig(FIG_DIR / "cvm_distance_landscape.svg")

rho_hat_cvm = rhos_dense[np.argmin(Sn)]
resid = Cn - gaussian_copula_cdf(Uu, Vv, rho_hat_cvm)
plt.figure(figsize=(5.2, 4.5))
im = plt.imshow(resid, origin="lower", extent=[0,1,0,1], aspect="equal")
plt.xlabel("u"); plt.ylabel("v"); plt.title(f"CvM residuals at $\\hat\\rho\\approx{rho_hat_cvm:.2f}$")
plt.colorbar(im, fraction=0.046, pad=0.04)
savefig(FIG_DIR / "cvm_residual_diagnostics.svg")